In [ ]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
import glob
import cv2
import os
import re
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
def plotPoke(x):
    f, a = plt.subplots(2, 8, figsize=(8, 2))
    for i in range(8):
        a[0][i].imshow(x[i])
        a[0,i].axis('off')
        a[1][i].imshow(x[i+8])
        a[1,i].axis('off')
    f.show()
    plt.draw()

In [ ]:
# Create an empty array to store pokemon pics
orig_img = np.empty((0, 40, 40, 3), dtype='uint8')
# Load all images and append into orig_img
path = os.path.abspath("./PixelRNN.ipynb")
path = re.sub('[a-zA-Z\s.]+$', '', path)
for pic in glob.glob(path+'Pokemon/*.bmp'):
    img = cv2.imread(pic)
    # cv2 save img as BGR, plt shows img as RGB
    img = img[:,:,[2,1,0]]
    orig_img = np.append(orig_img, [img], axis=0)

# Use plt to show original images 
plotPoke(orig_img)

In [ ]:
# Flat all data to one dimension
X_flat = np.empty((0, 4800), dtype='float32')
for i in xrange(orig_img.shape[0]):
    X_flat = np.append(X_flat, [orig_img[i].reshape(4800)], axis=0)
    
print 'Original image shape:  {0}\nFlatted image shape:  {1}'.format(orig_img.shape, X_flat.shape)

In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 4800 # Pokemon data input (img shape: 40x40x3)
n_steps = 3 # timesteps (1 pixel [R, G, B] for one step)
n_hidden = 128 # hidden layer num of features

# tf Graph input
X = tf.placeholder("float", [None, n_steps, n_input])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_steps]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_steps]))
}

In [ ]:
def yReshape(x):
    x = np.transpose(x, [1, 0, 2])
    x = np.reshape(x, [-1, n_input])
    return np.split(0, n_steps, x)

In [ ]:
orig_img[0].shape

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

def yReshape(x):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    return tf.split(0, n_steps, x)
    
pred = RNN(X, weights, biases)
y_true = tf.placeholder("float", [None, n_steps, n_input])
# Define loss and optimizer
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
x = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])

In [ ]:
x